In [1]:
import redcap
import pandas as pd
data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

proj = redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

In [2]:
crf_name = 'ironsupplement'

# Use the export_records() function to export the specified CRF
crf_data = proj.export_records(forms=[crf_name])

crf_data = pd.DataFrame(crf_data)
counts = (
    (crf_data['iron_supferrouspri_q2'] == '1') |
    (crf_data['ip_administer'] == '1')
).groupby(crf_data['cons_recordid_q1']).sum().rename('Total_ips')

counts2 = (
    (crf_data['iron_supferrouspri_q2'] == '1') &
    (crf_data['ip_administer'] == '1')
).groupby(crf_data['cons_recordid_q1']).sum().rename('Total_ips_both')

total_counts = counts + counts2
total_counts.name = 'Total_ips_combined'

crf_data = pd.merge(crf_data, total_counts, on=['cons_recordid_q1'], how='inner')
crf_data['total_IP_issued'] = 84
crf_data['balance'] = crf_data['total_IP_issued'] - crf_data['Total_ips_combined']

# Calculate the count of each instrument
instrument_counts = crf_data['redcap_repeat_instrument'].value_counts()

# Find the instrument with the maximum count
max_count_instrument = instrument_counts.idxmax()

# Group by the highest instrument count
grouped_data = crf_data.groupby('redcap_repeat_instrument').filter(lambda x: x['redcap_repeat_instrument'].value_counts().max() == instrument_counts.max())

# Filter out cons_recordid_q1 values starting with "IDV"
grouped_datagrouped_data = grouped_data[~grouped_data['cons_recordid_q1'].str.startswith('IDV')]

# Save the grouped data to a CSV file

In [3]:
sysmex_V12=proj.export_records(forms=['sysmexresult'])
sysmex_V12=pd.DataFrame(sysmex_V12)
sysmex_V12=sysmex_V12[
    (sysmex_V12['sysmexresult_complete']=='2')&
    (sysmex_V12['redcap_event_name']=='v12_arm_1')
    ]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [33]:
filter_data=grouped_data[grouped_data['cons_recordid_q1'].isin(sysmex_V12['cons_recordid_q1'])]
filter_data=pd.DataFrame(filter_data)

filter_data.to_csv('filter_data.csv',index=False)

In [ ]:
import json
import redcap
import pandas as pd
import pyodbc

data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

proj = redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

with open('config.json') as config_file:
    config = json.load(config_file)

username = config['username']
password = config['password']
server = config['server']
database = config['database']
driver = config['driver']

cnxn = pyodbc.connect(f"Driver={driver};Server={server};Database={database};uid={username};pwd={password};Trusted_Connection=yes;")
sql_query = "SELECT * FROM vw_IDeA_sysmex WHERE sysmexresult_complete != 2"  # Filter out completed forms
sysmexresult_data = pd.read_sql(sql_query, cnxn)
sysmexresult_data['sysmexresult_complete'] = 2
records_to_import = sysmexresult_data.to_dict('records')
event_name = 'v1_arm_1'

for record in records_to_import:
    record['redcap_event_name'] = event_name

response = proj.import_records(records_to_import, return_content='ids', overwrite=False)

# Check if the response contains any errors
if 'error' in response:
    print("There was an error in importing the data.")
else:
    print("Data has been successfully imported.")


In [ ]:
import json
import redcap
import pandas as pd
import pyodbc

data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

proj = redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

with open('config.json') as config_file:
    config = json.load(config_file)

username = config['username']
password = config['password']
server = config['server']
database = config['database']
driver = config['driver']

cnxn = pyodbc.connect(f"Driver={driver};Server={server};Database={database};uid={username};pwd={password};Trusted_Connection=yes;")

complete_sysmex=proj.export_records(forms=['sysmexresult'])
complete_sysmex=pd.DataFrame(complete_sysmex)
complete_sysmex=complete_sysmex
[
    (complete_sysmex['sysmexresult_complete']=='2')&
    (complete_sysmex['redcap_event_name']=='v1_arm_1')
    ]

sql_query = "SELECT * FROM vw_IDeA_sysmex"  # Retrieve all records from the source
sysmexresult_data = pd.read_sql(sql_query, cnxn)

# Filter out completed form IDs from your data
sysmexresult_data = sysmexresult_data[~sysmexresult_data['record_id'].isin(complete_sysmex)]

sysmexresult_data['sysmexresult_complete'] = 2
records_to_import = sysmexresult_data.to_dict('records')
event_name = 'v1_arm_1'

for record in records_to_import:
    record['redcap_event_name'] = event_name

response = proj.import_records(records_to_import, return_content='ids', overwrite=False)  # do not allow overiding

# Check if the response contains any errors
if 'error' in response:
    print("There was an error in importing the data.")
else:
    print("Data has been successfully imported.")
